# Домашнее задание №2

In [155]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer

## Task 1

### Загрузила данные, посмотрела на классы

In [170]:
path = 'SMSSpamCollection'
messages = pandas.read_csv(path, sep='\t', names=["label", "message"])

In [171]:
# print(messages)
print(messages.groupby('label').describe())

                                                        message
label                                                          
ham   count                                                4825
      unique                                               4516
      top                                Sorry, I'll call later
      freq                                                   30
spam  count                                                 747
      unique                                                653
      top     Please call our customer service representativ...
      freq                                                    4


In [59]:
# messages['length'] = messages['message'].map(lambda text: len(text))
# print(messages.head())

### Нормализовала данные: со знаками препинания. но! убрала штуки типа 'слово'+'..'

In [67]:
# 'слово'+'..'  плохо токенизируется функцией word_tokenize, поэтому уберем это
messages['message'] = [re.sub('(\w)\.\. ', '\\1 ', msg) for msg in messages['message']]
# print(messages.head())

### Нормализовала данные: без знаков препинания

In [83]:
messages['message'] = [re.sub('[!"?/\\().:;,-]', '', msg) for msg in messages['message']]

### Нормализовала данные:

In [144]:
lancaster_stemmer = LancasterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
messages['message'] = [re.sub('(\w)\.\. ', '\\1 ', msg) for msg in messages['message']]
messages['message'] = [word_tokenize(msg) for msg in messages['message']]

### стемматизация

In [136]:
arr = []
for msg in messages['message']:
    msg = [lancaster_stemmer.stem(i.lower()) for i in msg]
    arr.append(' '.join(msg))
messages['message'] = arr

In [137]:
print(messages.head())

  label                                            message
0   ham  go until jurong point , crazy avail on in bug ...
1   ham                        ok lar ... jok wif u on ...
2  spam  fre entry in 2 a wkly comp to win fa cup fin t...
3   ham  u dun say so ear hor ... u c already then say ...
4   ham  nah i do n't think he goe to usf , he liv arou...


### лемматизация

In [145]:
arr = []
for msg in messages['message']:
    msg = [wordnet_lemmatizer.lemmatize(i.lower()) for i in msg]
    arr.append(' '.join(msg))
messages['message'] = arr

In [146]:
print(messages.head())

  label                                            message
0   ham  go until jurong point , crazy available only i...
1   ham                    ok lar ... joking wif u oni ...
2  spam  free entry in 2 a wkly comp to win fa cup fina...
3   ham  u dun say so early hor ... u c already then sa...
4   ham  nah i do n't think he go to usf , he life arou...


### Нормализовала данные:  удаление стоп-слов

In [167]:
stopset = set(stopwords.words('english'))

In [172]:
messages['message'] = [re.sub('(\w)\.\. ', '\\1 ', msg) for msg in messages['message']]
messages['message'] = [[w for w in word_tokenize(msg) if not w in stopset] for msg in messages['message']]

arr = []
for msg in messages['message']:
    msg = [w for w in msg if not w in stopset]
    arr.append(' '.join(msg))
messages['message'] = arr

Go jurong point , crazy Available bugis n great world la e buffet ... Cine got amore wat ...
Ok lar ... Joking wif u oni ...
Free entry 2 wkly comp win FA Cup final tkts 21st May 2005 . Text FA 87121 receive entry question ( std txt rate ) T & C 's apply 08452810075over18 's
U dun say early hor ... U c already say ...
Nah I n't think goes usf , lives around though
FreeMsg Hey darling 's 3 week 's word back ! I 'd like fun still ? Tb ok ! XxX std chgs send , £1.50 rcv
Even brother like speak . They treat like aids patent .
As per request 'Melle Melle ( Oru Minnaminunginte Nurungu Vettam ) ' set callertune Callers . Press *9 copy friends Callertune
WINNER ! ! As valued network customer selected receivea £900 prize reward ! To claim call 09061701461 . Claim code KL341 . Valid 12 hours .
Had mobile 11 months ? U R entitled Update latest colour mobiles camera Free ! Call The Mobile Update Co FREE 08002986030
I 'm gon na home soon n't want talk stuff anymore tonight , k ? I 've cried enough 

### Построила модель

*пы.сы. я запускаю каждый кусок нормализации данных отдельно вместе с запуском модели*

In [173]:
bow = CountVectorizer()
bow.fit_transform(messages['message'])
# print(bow.vocabulary_)

<5572x8690 sparse matrix of type '<class 'numpy.int64'>'
	with 52738 stored elements in Compressed Sparse Row format>

In [174]:
bowed_messages = bow.transform(messages['message'])

In [175]:
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, messages['label'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [176]:
cv_results = cross_val_score(naive_model, bowed_messages, messages['label'], cv=10, scoring='accuracy')
print(cv_results.mean(), cv_results.std())

0.983130280394 0.00255807309826


1. Выборка не сбалансирована. ham приблизительно в 6 раз больше, чем spam. Так что попробуем справиться с этим при помощи кроссвалидации, вот.
2. Какой какой.. плохой очевидно 
3. разная токенизация (для простоты я смотрю на среднее значение, т.е. чем оно больше, тем лучше работает модель):
    * **со знаками препинания:** mean: 0.980260320614    std: 0.00488300101375
    * **без знаков препинания:** mean: 0.977745560545    std: 0.00522292832323
    * **стемматизация + знаки препинания:** mean: 0.978646452155    std: 0.00411601196134
    * **лемматизация + знаки препинания:** mean: 0.97900390639    std: 0.00488611860617
    * **удаление стоп-слов:** mean: 0.983130280394    std: 0.00255807309826